from https://www.youtube.com/watch?v=CtzfbUwrYGI

This notebook allowed me to build and train a simple cnn model to learn pytorch

In [1]:
# %pip install numpy pillow
# python.exe -m pip install --upgrade pip setuptools wheel
# pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu130

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='../../data/CIFAR10', train=True,
                                          download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='../../data/CIFAR10', train=False,
                                          download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
                                           shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32,
                                          shuffle=False, num_workers=0)

c:\Users\FlowUP\Documents\Projets\Eye-Disease-Classifier\.venv\Lib\site-packages\torchvision\datasets\cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


In [4]:
image, label = train_data[0]
print(f'Image shape: {image.shape}, Label: {label}')

Image shape: torch.Size([3, 32, 32]), Label: 6


In [5]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [15]:
NUMBER_OF_CLASSES = 10

class NeuralNet(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5, padding=1) # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5, padding=1) # (24, 12, 12)
        self.pool = nn.MaxPool2d(2, 2) # (24, 6, 6)
        self.fc1 = nn.Linear(24 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84) 
        self.fc3 = nn.Linear(84, NUMBER_OF_CLASSES) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [16]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [17]:
for epoch in range(30):  # loop over the dataset multiple times
    print(f'Training epoch {epoch}...')

    runing_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step() # update weights using gradients of learning rate size
        runing_loss += loss.item()
    print(f'Loss: {runing_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.2177
Training epoch 1...
Loss: 1.7803
Training epoch 2...
Loss: 1.5107
Training epoch 3...
Loss: 1.3795
Training epoch 4...
Loss: 1.2732
Training epoch 5...
Loss: 1.1831
Training epoch 6...
Loss: 1.1029
Training epoch 7...
Loss: 1.0361
Training epoch 8...
Loss: 0.9758
Training epoch 9...
Loss: 0.9233
Training epoch 10...
Loss: 0.8748
Training epoch 11...
Loss: 0.8274
Training epoch 12...
Loss: 0.7856
Training epoch 13...
Loss: 0.7492
Training epoch 14...
Loss: 0.7115
Training epoch 15...
Loss: 0.6802
Training epoch 16...
Loss: 0.6456
Training epoch 17...
Loss: 0.6171
Training epoch 18...
Loss: 0.5844
Training epoch 19...
Loss: 0.5536
Training epoch 20...
Loss: 0.5256
Training epoch 21...
Loss: 0.4948
Training epoch 22...
Loss: 0.4725
Training epoch 23...
Loss: 0.4409
Training epoch 24...
Loss: 0.4144
Training epoch 25...
Loss: 0.3947
Training epoch 26...
Loss: 0.3674
Training epoch 27...
Loss: 0.3415
Training epoch 28...
Loss: 0.3194
Training epoch 29...
Los

In [ ]:
torch.save(net.state_dict(), '../Models/Simple_CNN_Discovery.pth')
print('Finished Training')

Finished Training


In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('Simple_CNN_Discovery.pth'))

<All keys matched successfully>

In [21]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')

Accuracy of the network on the 10000 test images: 68.87%


In [22]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


def load_image(image_path):
    image = Image.open(image_path) # .convert('RGB')
    image = new_transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

image_path = ['test_image.jpg']  # Replace with your image path
image = [load_image(path) for path in image_path]

net.eval()
with torch.no_grad():
    for img in image:
        output = net(img)
        _, predicted = torch.max(output, 1)
        class_index = predicted.item()
        class_name = class_names[class_index]
        print(f'Predicted class: {class_name}')

Predicted class: dog
